https://www.kaggle.com/c/bike-sharing-demand/data?select=train.csv
# 자전거 대여량 예측 모델

datetime - 시간별 날짜 + 타임스탬프   
season - 1 = 봄, 2 = 여름, 3 = 가을, 4 = 겨울   
holiday - 해당 요일이 휴일인지 여부   
workingday - 해당 요일이 주말이나 휴일이 아닌지 여부    
weather - 1: 맑음, 구름 약간, 부분적으로 구름, 부분적으로 구름   
2: 안개 + 구름, 안개 + 깨진 구름, 안개 + 구름 약간, 안개   
3: 가벼운 눈, 가벼운 비 + 뇌우 + 흩어진 구름, 가벼운 비 + 흩어진 구름   
4: 폭우 + 얼음판 + 뇌우 + 안개, 눈 + 안개    
temp - 섭씨 온도   
atemp - 체감 온도   
섭씨 humid - 상대 습도   
windspeed - 풍속   
casual - 미등록된 사용자가 시작한 대여 수   
registered - 등록된 사용자가 시작한 대여 수   
count - 총 대여 수   

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

In [53]:
df = pd.read_csv("BikeSharingDemand/train.csv")
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [63]:
import datetime
df["datetime"] = pd.to_datetime(df["datetime"])
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [67]:
# 제거

print(sum(df["casual"]) + sum(df["registered"]) == sum(df["count"]))
df = df.drop(["casual", "registered"], axis=1)


True


In [68]:
df = df.drop("workingday", axis=1)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   10886 non-null  object 
 1   season     10886 non-null  int64  
 2   holiday    10886 non-null  int64  
 3   weather    10886 non-null  int64  
 4   temp       10886 non-null  float64
 5   atemp      10886 non-null  float64
 6   humidity   10886 non-null  int64  
 7   windspeed  10886 non-null  float64
 8   count      10886 non-null  int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 765.5+ KB


In [30]:
df.describe()

,season,holiday,weather,temp,atemp,humidity,windspeed,count
count,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,1.418427,20.23086,23.655084,61.886460,12.799395,191.574132
std,1.116174,0.166599,0.633839,7.79159,8.474601,19.245033,8.164537,181.144454
min,1.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,42.000000
50%,3.000000,0.000000,1.000000,20.50000,24.240000,62.000000,12.998000,145.000000
75%,4.000000,0.000000,2.000000,26.24000,31.060000,77.000000,16.997900,284.000000
max,4.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,977.000000


In [31]:
# for i in df.drop("count", axis=1).columns:
#     print(i)
#     plt.scatter(x= df["count"], y=df[i])
#     plt.legend(i)
#     plt.show()

In [32]:
def poly_func(x, y, model):
    results = []
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=142)
    for degree in range(1, 5):
        model_poly = Pipeline(
            [
                ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
                ('Linear', model())
            ]
        )
        model_poly.fit(x_train, y_train)
        pred_poly = model_poly.predict(x_test)
        
        mse = mean_squared_error(y_test, pred_poly)
        rmse = np.sqrt(mse)

        r2 = r2_score(y_test, pred_poly)

        results.append(
            {
                'degree': degree,
                'MSE': mse,
                "RMSE": rmse,
                "r2": r2
            }
        )


    return pd.DataFrame(results)

In [33]:
def model_func(x, y, model, *kwargs):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=142)
    md = None
    if kwargs:
        md = model(*kwargs)
    else:
        md = model()
        
    md.fit(x_train, y_train)
    y_pred = md.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    nmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    nmse_cv = cross_val_score(md, x, y, scoring='neg_mean_squared_error', cv=5)
    r2_cv = cross_val_score(md, x, y, scoring='r2', cv=5)

    return pd.DataFrame(
                            {
                                "r2": [r2, np.mean(r2_cv)],
                                "neg_mean_squared_error": [nmse, -np.mean(nmse_cv)]
                            },
                            index=["score", "cross_val_score"]
                        )


In [34]:
def Linear_model_func(x, y, model):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=142)
    lr = model()
    lr.fit(x_train, y_train)
    y_pred = lr.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    nmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    nmse_cv = cross_val_score(lr, x, y, scoring='neg_mean_squared_error', cv=5)
    r2_cv = cross_val_score(lr, x, y, scoring='r2', cv=5)

    return pd.DataFrame(
                            {
                                "r2": [r2, np.mean(r2_cv)],
                                "neg_mean_squared_error": [nmse, -np.mean(nmse_cv)]
                            },
                            index=["score", "cross_val_score"]
                        )


--------------
# 

## datetime 월-일-시-분

In [35]:
df_all = pd.DataFrame(df)

In [36]:
# 월 일 시 분 만 남기기
def remove_year_sceond(data):
    return data[5:-4]

le = LabelEncoder()
df_all["datetime"] = df_all["datetime"].apply(remove_year_sceond)
df_all["datetime"] = le.fit_transform(df_all["datetime"])

df_all

,datetime,season,holiday,weather,temp,atemp,humidity,windspeed,count
0,0,1,0,1,9.84,14.395,81,0.0000,16
1,1,1,0,1,9.02,13.635,80,0.0000,40
2,2,1,0,1,9.02,13.635,80,0.0000,32
3,3,1,0,1,9.84,14.395,75,0.0000,13
4,4,1,0,1,9.84,14.395,75,0.0000,1
...,...,...,...,...,...,...,...,...,...
10881,5467,4,0,1,15.58,19.695,50,26.0027,336
10882,5468,4,0,1,14.76,17.425,57,15.0013,241
10883,5469,4,0,1,13.94,15.910,61,15.0013,168
10884,5470,4,0,1,13.94,17.425,61,6.0032,129


In [37]:
# 전체 데이터에 관한

X = df_all.drop("count", axis=1).values
y = df_all["count"].values

In [38]:
linearRegression_all = Linear_model_func(X, y, LinearRegression)
linearRegression_all

,r2,neg_mean_squared_error
score,0.257731,155.195923
cross_val_score,-0.073074,28320.809754


In [39]:
model_func(X, y, RidgeCV, [0.001, 0.01, 0.1, 1, 10, 100])

,r2,neg_mean_squared_error
score,0.257931,155.175002
cross_val_score,-0.071500,28293.799602


In [40]:
model_func(X, y, LassoCV)

,r2,neg_mean_squared_error
score,0.255533,155.425525
cross_val_score,-0.077324,28513.196684


-------------
## 월별 회귀 결과 예측

In [41]:
df_month = pd.DataFrame(df)

In [42]:
# 월별
# 월 만 남기기
def remove_ydhms(data):
    return data[5:7]
df_month["datetime"] = df_month["datetime"].apply(remove_ydhms)
df_month

,datetime,season,holiday,weather,temp,atemp,humidity,windspeed,count
0,01,1,0,1,9.84,14.395,81,0.0000,16
1,01,1,0,1,9.02,13.635,80,0.0000,40
2,01,1,0,1,9.02,13.635,80,0.0000,32
3,01,1,0,1,9.84,14.395,75,0.0000,13
4,01,1,0,1,9.84,14.395,75,0.0000,1
...,...,...,...,...,...,...,...,...,...
10881,12,4,0,1,15.58,19.695,50,26.0027,336
10882,12,4,0,1,14.76,17.425,57,15.0013,241
10883,12,4,0,1,13.94,15.910,61,15.0013,168
10884,12,4,0,1,13.94,17.425,61,6.0032,129


In [43]:
mX = df_month.drop("count", axis=1).values
my = df_month["count"].values

In [44]:
linearRegression_month = Linear_model_func(mX, my, LinearRegression)
linearRegression_month

,r2,neg_mean_squared_error
score,0.257611,155.208436
cross_val_score,-0.070141,28302.825122


In [45]:
model_func(mX, my, Ridge)

,r2,neg_mean_squared_error
score,0.257614,155.208128
cross_val_score,-0.070122,28302.486214


In [46]:
model_func(mX, my, Lasso)

,r2,neg_mean_squared_error
score,0.257852,155.183250
cross_val_score,-0.062883,28169.278795


In [47]:
model_func(mX, my, RidgeCV, [0.001, 0.01, 0.1, 1, 10, 100])

,r2,neg_mean_squared_error
score,0.257809,155.187667
cross_val_score,-0.068558,28274.572526


In [48]:
model_func(mX, my, LassoCV)

,r2,neg_mean_squared_error
score,0.257809,155.187738
cross_val_score,-0.108024,28498.708024


In [49]:
model_func(mX, my, ElasticNet)

,r2,neg_mean_squared_error
score,0.257625,155.206938
cross_val_score,-0.056576,28052.176813


In [50]:
for i in [Ridge, LinearRegression, RandomForestRegressor, Lasso]:
    print(i.__name__)
    print(poly_func(mX, my, i))

Ridge
   degree           MSE        RMSE        r2
0       1  24089.562885  155.208128  0.257614
1       2  22771.141643  150.901099  0.298245
2       3  21893.342381  147.963990  0.325296
3       4  22110.358064  148.695521  0.318608
LinearRegression


c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=5.93941e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


   degree           MSE        RMSE        r2
0       1  24089.658654  155.208436  0.257611
1       2  22773.843208  150.910050  0.298161
2       3  21840.921107  147.786742  0.326912
3       4  23043.431617  151.800631  0.289853
RandomForestRegressor
   degree           MSE        RMSE        r2
0       1  22569.754938  150.232337  0.304451
1       2  22433.338778  149.777631  0.308655
2       3  22070.070024  148.559988  0.319850
3       4  21969.209452  148.220138  0.322958
Lasso


c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.105e+07, tolerance: 2.512e+04
  model = cd_fast.enet_coordinate_descent(
c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.417e+07, tolerance: 2.512e+04
  model = cd_fast.enet_coordinate_descent(


   degree           MSE        RMSE        r2
0       1  24081.841168  155.183250  0.257852
1       2  22832.279324  151.103538  0.296360
2       3  22285.293479  149.282596  0.313217
3       4  21941.950370  148.128155  0.323798


c:\Users\Admin\miniconda3\envs\hi_ml_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.260e+07, tolerance: 2.512e+04
  model = cd_fast.enet_coordinate_descent(


---------------